In [11]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [ ]:
class GAN():
    
    def __init__(self):
        
        self.axes = 3
        self.length = 500
        self.data_shape = (self.length, self.axes)
        
        self.latent_dim = 100
        
        optimizer = Adam()
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = 'binary_crossentropy', 
                                   optimizer = optimizer, 
                                   metrics = ['accuracy'])
        
        self.generator = self.build_generator()
        
        z = Input(shape = self.latent_dim)
        acc = self.generator(z)
        
        self.discriminator.trainable = False
        
        validity = self.discriminator(acc)
        
        self.combined = Model(z, validity)
        self.combined.compile(loss = 'binary_crossentropy', optimizer = optimizer)
        
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(LSTM(256, input_shape = (500, 3)))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(LSTM(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.summary()
        
        acc = Input()
        validity = model(acc)
        
        return Model(acc, validity)
    
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256, input_dim = self.latent_dim))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(BatchNormalization(momentum = 0.8))
        model.add(Dense(np.prod(self.data_shape), activation = 'tanh'))
        model.add(Reshape(self.data_shape))
        
        model.summary()
        
        noise = Input(shape = self.latent_dim)
        acc = model(noise)
        
        return Model(noise, acc)

    def train(self, epochs, batch_size = 128, sample_interval = 50):
        
        (X_train, _), (_, _) = mnist.load_data()
        
        X_train = X_train / 127.5 - 1
        X_train = np.expand_dims(X_train, axis = 3)